In [78]:
from PIL import Image
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import interact
import ipywidgets as widgets
%matplotlib inline


# II. Second equation:   

### $$\partial^2_x f(x,y) + \partial^2_y f(x,y) - \gamma f(x,y) = 0 $$


Define $\beta = \ln (\gamma + 1)$. For every $(i,j) \in \mathcal{D}$, let $(X_n)_{n \in \mathbb{N}}$ be the two-dimensional random walk initially in $(i,j)$, such that for every $n \in \mathbb{N}$:
* if $X_n = (k,l)$ is on the border $\partial \mathcal{D}$, the walk stays put with probability $1$: $\mathbb{P}[X_{n+1} = (k,l)] = 1$.
* else,  $X_n = (k,l) \in \mathcal{D} \setminus \partial \mathcal{D}$, we chose a neighbour of $(k,l)$ with uniform probability:
 * $\mathbb{P}[X_{n+1} = (k+1,l)] = 1/4$
 * $\mathbb{P}[X_{n+1} = (k-1,l)] = 1/4$
 * $\mathbb{P}[X_{n+1} = (k,l+1)] = 1/4$
 * $\mathbb{P}[X_{n+1} = (k,l-1)] = 1/4$

Let $T = \inf \{ n \in \mathbb{N}, X_n \in \partial \mathcal{D} \}$ be the first time the border of $\mathcal{D}$ is reached. Consider the function $v : \mathcal{D} \to \mathbb{R}$ defined on every $(i,j) \in \mathcal{D}$ by: 
### $$v(i,j) = \mathbb{E}_i\big[\phi(X_T) exp \big( - \beta \sum\limits^{T}_{j = 0} \mathbb{1}_{\{ X_j \notin \partial{\mathbb{D}} \}}\big)\big]$$

Since the walk is symmetric, conditionning with respect to the first step yields for every $(i,j) \notin \partial \mathbb{D}$:

\begin{equation*}
    \begin{aligned}
        v(i,j) &= \frac {1} {4} \mathbb{E} \big[ \phi(X_T)exp \big( - \beta \sum_\limits{i = 0}^{T} \mathbb{1}_{X_i \notin \{0;L \} }\big) | X_0 = (i,j), X_1 = (i+1, j)  \big] \\
 &+ \frac {1} {4} \mathbb{E} \big[ \phi(X_{T})exp \big( - \beta \sum_\limits{i = 0}^{T} \mathbb{1}_{X_i \notin \{0;L \} }\big) | X_0 = (i,j), X_1 = (i-1, j)  \big] \\
 &+ \frac {1} {4} \mathbb{E} \big[ \phi(X_{T})exp \big( - \beta \sum_\limits{i = 0}^{T} \mathbb{1}_{X_i \notin \{0;L \} }\big) | X_0 = (i,j), X_1 = (i, j+1)  \big] \\
 &+ \frac {1} {4} \mathbb{E} \big[ \phi(X_{T})exp \big( - \beta \sum_\limits{i = 0}^{T} \mathbb{1}_{X_i \notin \{0;L \} }\big) | X_0 = (i,j), X_1 = (i, j-1)  \big]
    \end{aligned}
\end{equation*}

Then Markov's property implies that the sequence $(X_{n+1})_{n \in \mathbb{N}}$ is also a Markov chain with same transition as $(X_{n})_{n \in \mathbb{N}}$, and with initial position $X_1$. Therefore:
$$v(i,j) = \frac {e^{-\beta}} {4} [v(i+1,j) + v(i,j+1) + v(i-1,j) + v(i,j-1)]$$

 $$\frac {e^{-\beta}} {4} [v(i+1,j) + v(i,j+1) + v(i-1,j) + v(i,j-1) - 4 v(i,j)] - (1 - e^{-\beta})v(i,j) = 0$$


Let $\tilde{\Delta}$ be the discretised Laplacian, sending every function $f: \mathbb{D} \to \mathbb{R}$ to the function $$\tilde{\Delta}f : (i,j) \mapsto \frac 1 4 \sum\limits_{(k,l) ~(i,j)} [f(k,l) - f(i,j)]$$

The previous equality can therefore be rewritten as follows:

$$\tilde{\Delta}v(i,j)  - (e^{\beta} - 1)v(i,j) = 0$$


$v_n$ therefore checks a discretised version of the following equation:
$$\partial_t f(x,y,t) = e^{-\beta}\Delta f(x,y,t) - (1 - e^{-\beta}) f(x,y,t)$$


When $t \to \infty$, regular solutions reach their stationary values, hence $\partial_t f(x,y,t) \approx 0$. Notice that we chose $\beta$ so that $\gamma = e^{\beta} - 1$. Therefore for large enough values of $n$, $v_n(i)$ approximates the value in $i$ of the solution of the equation:
### $$\Delta f(x,y,t) - \gamma f(x,y,t) =0 $$


# Code for time dependent equation

In [79]:
#! /usr/bin/env python

import random
import numpy as np
import time

In [80]:
# define phi as f is at time t = 0 - in particular, phi defines border conditions
def phi(x, y, L):
    if x == 0 or x == L:
        return 1
    return 0
vphi = np.vectorize(phi)

In [81]:
def v_n(init, L, beta, n, K):
    somme = 0
    for _ in range(K):
        x = init[0]
        y = init[1]
        coordList = [[x,y]]
        number_non_touches = 0
        for j in range(n):
            coordList.append([x,y])
            if x != 0 and y != 0 and y != L and x != L: # in that case the walk's fate is sealed
                number_non_touches += 1
                rand = random.random()
                # Go with equal probability left, up, right or down
                if 0 <= rand < 0.25:
                    x -= 1
                elif 0.25 <= rand < 0.5:
                    x += 1
                elif 0.5 <= rand < 0.75:
                    y -= 1
                else:
                    y += 1
                
        somme += vphi(np.array([coord[0] for coord in coordList]),np.array([coord[1] for coord in coordList]), L)*np.exp(- beta*np.arange(n+1))
    return somme/K


In [82]:
K = 50
L = 10
N = 900
beta = 1 


In [83]:
arrayImages = np.zeros((N,L,L))
for i in range(L):
    for j in range(L):
        ls = v_n([i,j], L, beta, N, K)
        for n in range(N):
            arrayImages[n][i][j] = ls[n]

In [87]:
%matplotlib inline
import matplotlib.pyplot as plt
def _show(frame=widgets.IntSlider(min=0,max=len(arrayImages) - 1,step=1,value=0)):
    fig, ax = plt.subplots(figsize=(7,7))
    ax.imshow(arrayImages[frame][:][:].T, origin="lower",
               extent=[0, 1, 0, 1], cmap="jet")
interact(_show)

interactive(children=(IntSlider(value=0, description='frame', max=899), Output()), _dom_classes=('widget-inter…

<function __main__._show(frame=IntSlider(value=0, description='frame', max=899))>